# Convert fore names to simplified (one word) fore names

In [1]:
import collections
import string
import pandas

import pubmedpy.names

In [2]:
name_df_paths = [
    'data/pubmed/authors.tsv.xz',
    'data/pmc/authors.tsv.xz',
    'data/iscb/keynotes.tsv',
]
path_to_df = {}
for path in name_df_paths:
    name_df = pandas.read_csv(path, keep_default_na=False, na_values=[''], sep='\t')
    path_to_df[path] = name_df

In [3]:
def simplify_names(dfs, name_column):
    names = pandas.concat(df[name_column] for df in dfs)
    simple_name_df = names.value_counts(dropna=True).reset_index()
    simple_name_df.columns = [name_column, 'n_authors']
    assert simple_name_df[name_column].isna().sum() == 0
    simplifier = getattr(pubmedpy.names, f'simplify_{name_column}')
    simple_name_df[f'{name_column}_simple'] = simple_name_df[name_column].map(lambda x: simplifier(x, lower=True))
    simple_name_df = simple_name_df.sort_values(name_column)
    return simple_name_df

In [4]:
simple_fore_name_df = simplify_names(path_to_df.values(), 'fore_name')
simple_fore_name_df

,fore_name,n_authors,fore_name_simple
36676,(Max) Zong-Ming,1,max
34129,", Yuxin",1,yuxin
24,A,374,None
2285,A A,17,None
25343,A Aldo,2,aldo
...,...,...,...
19185,Špela,2,špela
21984,Žiga,2,žiga
32158,Živa,1,živa
14202,Živadin,2,živadin


In [5]:
simple_last_name_df = simplify_names(path_to_df.values(), 'last_name')
simple_last_name_df

,last_name,n_authors,last_name_simple
35242,Michor,1,michor
33842,Rozowsky,1,rozowsky
33851,Slater,1,slater
36668,'t Hart,1,t hart
2507,'t Hoen,14,t hoen
...,...,...,...
32597,Žárský,2,žárský
34477,železný,1,železný
37393,žurauskienė,1,žurauskienė
17160,‘t Hoen,2,‘t hoen


In [6]:
simple_fore_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,fore_name,n_authors,fore_name_simple
3509,Marine,11,marine
6930,Guy N,6,guy
8742,Moliang,4,moliang
12014,Denisa,3,denisa
24033,Cátia,2,cátia
22001,Amy C,2,amy
21178,Reiko Matsuda,2,reiko
18523,Laxmikanth,2,laxmikanth
16291,Bo Kyung,2,bo
14534,Wendy T,2,wendy


In [7]:
simple_last_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,last_name,n_authors,last_name_simple
1196,Lou,23,lou
1672,van Hijum,18,van hijum
3317,Parks,11,parks
4474,Zarringhalam,9,zarringhalam
9281,Löwer,5,löwer
9666,Pascarella,4,pascarella
12877,Harkin,4,harkin
14026,Colsch,3,colsch
14311,Krupp,3,krupp
23003,Hagenaars,2,hagenaars


In [8]:
# Most common last names
(
    simple_last_name_df
    .groupby('last_name_simple')
    .n_authors.sum()
    .sort_values(ascending=False)
    .reset_index()
    .head(3)
)

,last_name_simple,n_authors
0,wang,3240
1,li,2715
2,zhang,2655


In [9]:
simple_fore_name_df.to_csv('data/names/fore-names.tsv.xz', sep='\t', index=False)
simple_last_name_df.to_csv('data/names/last-names.tsv.xz', sep='\t', index=False)